In [2]:
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import sort_array, broadcast, collect_list, collect_set,struct, array_sort, explode, posexplode, create_map,from_utc_timestamp, from_unixtime, to_date, sum,count, mean, countDistinct, round
from pyspark.sql.functions import year, month, dayofmonth
from pyspark.sql.types import IntegerType,StructField,StructType, LongType, FloatType, MapType, DateType,StringType,TimestampType,ArrayType
from pyspark.sql.functions import col, when, regexp_replace
import pyspark.sql.functions as f

from pyspark.sql.functions import monotonically_increasing_id, coalesce, date_trunc, regexp_extract


import os


In [3]:
memory = "7g"
os.environ["PYSPARK_SUBMIT_ARGS"] = (
    ' --conf spark.driver.maxResultSize=' + memory + ' pyspark-shell'
)
os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'



In [4]:
spark = SparkSession \
    .builder \
    .getOrCreate()
spark.sparkContext.uiWebUrl



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/01/07 13:55:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'http://liris502-x003.insa-lyon.fr:4040'

In [5]:
outputs_file = spark.read.load("data/final/enrichedOutputs")

In [6]:
outputs_file.show(5)

25/01/07 13:56:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------+-----------+--------------------+--------------------+----+---------+----------+---------+----------+----------+----+-----+---+-----+---+--------+----------+-------------------+--------+--------------------+------------+------------+--------------------+------------+------------+--------------------+
|     id_dst|     id_src|                 dst|                 src|trID|output_id|     value|nb_inputs|nb_outputs| timestamp|year|month|day| bloc|fee|total_in| total_out|           PriceUSD|HashRate|            valueUSD|      src_cl|src_identity|src_identity_radical|      dst_cl|dst_identity|dst_identity_radical|
+-----------+-----------+--------------------+--------------------+----+---------+----------+---------+----------+----------+----+-----+---+-----+---+--------+----------+-------------------+--------+--------------------+------------+------------+--------------------+------------+------------+--------------------+
|       6641|       NULL|nonstandardf10dd3...|         

In [9]:
#count the number of unique blocs
outputs_file.select(countDistinct("bloc")).show()

+--------------------+
|count(DISTINCT bloc)|
+--------------------+
|              100000|
+--------------------+



In [10]:
#count the number of unique transactions
outputs_file.select(countDistinct("trID")).show()

+--------------------+
|count(DISTINCT trID)|
+--------------------+
|              216573|
+--------------------+



In [11]:
#coutn the number of unique addresses in src
outputs_file.select(countDistinct("src")).show()

+-------------------+
|count(DISTINCT src)|
+-------------------+
|              63108|
+-------------------+



In [12]:
outputs_file.select(countDistinct("id_src")).show()

+----------------------+
|count(DISTINCT id_src)|
+----------------------+
|                 63107|
+----------------------+



In [13]:
#count the number of different users in input
outputs_file.select(countDistinct("src_cl")).show()

+----------------------+
|count(DISTINCT src_cl)|
+----------------------+
|                 60244|
+----------------------+



In [14]:
#list all different users names
outputs_file.select("src_identity_radical").distinct().show()

+--------------------+
|src_identity_radical|
+--------------------+
|        CoinJoinMess|
|                NULL|
+--------------------+

